In [1]:
from sage.all import *
from sage.crypto.sbox import *
from sage.crypto.sboxes import sboxes
import numpy as np

In [8]:
# Modified from https://github.com/abrari/block-cipher-testing

# Works only on square s-boxes
def sbox_bic(sbox):

    maxCorr = 0
    m = len(sbox)
    for i in range(m):
        ei = 2**i 
        for j in range(m):
            for k in range(m):
                if j != k:
                    avalanche_vec_j = np.zeros(2**m, dtype=float)
                    avalanche_vec_k = np.zeros(2**m, dtype=float)

                    # Each possible input
                    for X in range(2**m):
                        ej = 2**j 
                        ek = 2**k 

                        dei = sbox[X] ^ sbox[X ^ ei]
                        dej = (dei & ej) >> j
                        dek = (dei & ek) >> k 

                        avalanche_vec_j[X] = dej 
                        avalanche_vec_k[X] = dek
                    corr = abs(np.corrcoef(avalanche_vec_j, avalanche_vec_k)[0,1])
                
                    if maxCorr < corr:
                        maxCorr = corr
    return maxCorr

sbox_bic(sboxes["AES"])


0.1341246360271594

In [10]:
# Modified version to get the average SAC of an sbox from:
#       https://github.com/abrari/block-cipher-testing
def avg_sac(sbox):
    m = len(sbox)
    sac_mat = np.array([np.zeros(m, dtype=float) for _ in range(m)])

    for i in range(m):
        ei = 2**i 
        for j in range(m):
            ej = 2**j 
            for X in range(2**m):
                dei = sbox[X] ^ sbox[X ^ ei]
                sac_mat[i][j] += (dei & ej) >> j 
    
    output_len = 2**m
    for i in range(m):
        for j in range(m):
            sac_mat[i][j] /= output_len

    sum = 0 
    for i in range(m):
        sum += np.sum(sac_mat[i])
    avg = sum / (m**2)
    return avg 

avg_sac(sboxes["AES"])

0.5048828125

In [2]:
def print_s_box_properties(S, S_name=None):
    if not S_name:
        S_name = "the s-box"
    nonlinearity_stmt = f"The nonlinearity of {S_name} is: {S.nonlinearity()}\n"
    linearity_prob_stmt = f"The linear probability of {S_name} is: {S.maximal_linear_bias_relative()}\n" 
    differential_prob_stmt = f"The differential probability of {S_name} is: {S.maximal_difference_probability()}\n" 
    print(f"{nonlinearity_stmt}{linearity_prob_stmt}{differential_prob_stmt}")

def generate_random_sbox():
    p = Permutations(range(256)).random_element()
    return SBox(*p)

In [4]:
nl_dict = {}
lp_dict = {}
dp_dict = {}
bic_dict = {}

for sbox in sboxes.values():
    if len(sbox) == 8:
        nl = sbox.nonlinearity()
        lp = sbox.maximal_linear_bias_relative()
        dp = sbox.maximal_difference_probability()
        if nl in nl_dict.keys():
            nl_dict[nl]+=1
        else:
            nl_dict[nl] = 1

        if lp in lp_dict.keys():
            lp_dict[lp] += 1
        else:
            lp_dict[lp] = 1

        if dp in dp_dict.keys():
            dp_dict[dp] += 1
        else:
            dp_dict[dp] = 1

print(nl_dict)
print(lp_dict)
print(dp_dict)

{112: 11, 94: 5, 102: 1, 100: 8, 96: 20, 0: 1, 88: 1, 64: 4, 104: 4, 90: 1, 82: 1, 92: 1}
{0.0625: 11, 0.1328125: 5, 0.1015625: 1, 0.109375: 8, 0.125: 20, 0.5: 1, 0.15625: 1, 0.25: 4, 0.09375: 4, 0.1484375: 1, 0.1796875: 1, 0.140625: 1}
{0.015625: 12, 0.03125: 17, 0.0390625: 11, 0.046875: 6, 0.5: 2, 0.0625: 6, 0.25: 4}


In [4]:
print(len(nl_dict))
print(len(lp_dict))
print(len(dp_dict))

12
12
7


In [5]:
sboxes["AES"]

(99, 124, 119, 123, 242, 107, 111, 197, 48, 1, 103, 43, 254, 215, 171, 118, 202, 130, 201, 125, 250, 89, 71, 240, 173, 212, 162, 175, 156, 164, 114, 192, 183, 253, 147, 38, 54, 63, 247, 204, 52, 165, 229, 241, 113, 216, 49, 21, 4, 199, 35, 195, 24, 150, 5, 154, 7, 18, 128, 226, 235, 39, 178, 117, 9, 131, 44, 26, 27, 110, 90, 160, 82, 59, 214, 179, 41, 227, 47, 132, 83, 209, 0, 237, 32, 252, 177, 91, 106, 203, 190, 57, 74, 76, 88, 207, 208, 239, 170, 251, 67, 77, 51, 133, 69, 249, 2, 127, 80, 60, 159, 168, 81, 163, 64, 143, 146, 157, 56, 245, 188, 182, 218, 33, 16, 255, 243, 210, 205, 12, 19, 236, 95, 151, 68, 23, 196, 167, 126, 61, 100, 93, 25, 115, 96, 129, 79, 220, 34, 42, 144, 136, 70, 238, 184, 20, 222, 94, 11, 219, 224, 50, 58, 10, 73, 6, 36, 92, 194, 211, 172, 98, 145, 149, 228, 121, 231, 200, 55, 109, 141, 213, 78, 169, 108, 86, 244, 234, 101, 122, 174, 8, 186, 120, 37, 46, 28, 166, 180, 198, 232, 221, 116, 31, 75, 189, 139, 138, 112, 62, 181, 102, 72, 3, 246, 14, 97, 53, 87, 18

In [7]:
print(sboxes["AES"].SAC())

AttributeError: 'SBox' object has no attribute 'SAC'

From sbox:
https://github.com/okazymyrov/sbox
